# Doordash Delivery Time Research


## Data Exploration


In [64]:
import pandas as pd

df = pd.read_csv("../Artifacts/data_ingestion/data.csv")
df.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
0,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0
1,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0
2,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0
3,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0
4,3.0,2015-01-28 20:30:38,2015-01-28 21:08:58,5477,NaN,1.0,3,5000,3,1500,1900,2.0,2.0,2.0,446,338.0


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197427 entries, 0 to 197426
Data columns (total 16 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   market_id                                     196440 non-null  float64
 1   created_at                                    197427 non-null  object 
 2   actual_delivery_time                          197420 non-null  object 
 3   store_id                                      197427 non-null  int64  
 4   store_primary_category                        192667 non-null  object 
 5   order_protocol                                196432 non-null  float64
 6   total_items                                   197427 non-null  int64  
 7   subtotal                                      197427 non-null  int64  
 8   num_distinct_items                            197427 non-null  int64  
 9   min_item_price                                19

In [66]:
df.isnull().sum()

market_id                                         987
created_at                                          0
actual_delivery_time                                7
store_id                                            0
store_primary_category                           4760
order_protocol                                    995
total_items                                         0
subtotal                                            0
num_distinct_items                                  0
min_item_price                                      0
max_item_price                                      0
total_onshift_dashers                           16262
total_busy_dashers                              16262
total_outstanding_orders                        16262
estimated_order_place_duration                      0
estimated_store_to_consumer_driving_duration      526
dtype: int64

In [67]:
df.shape

(197427, 16)

## Feature Engineering


In [ ]:
df["created_at"] = pd.to_datetime(df["created_at"])
df["actual_delivery_time"] = pd.to_datetime(df["actual_delivery_time"])
df["delivery_duration_minutes"] = (
    df["actual_delivery_time"] - df["created_at"]
).dt.total_seconds() / 60

In [ ]:
df["hour"] = df["created_at"].dt.hour
df["day_of_week_num"] = df["created_at"].dt.dayofweek
df["is_weekend"] = df["day_of_week_num"].isin([5, 6]).astype(int)

In [ ]:
df["total_busy_dashers"] = abs(df["total_busy_dashers"])  # Handle negative values
df["total_onshift_dashers"] = abs(df["total_onshift_dashers"])
df["dashers_per_order"] = df["total_onshift_dashers"] / (
    df["total_outstanding_orders"] + 1e-5
)
df["%_dashers_avail"] = df["total_busy_dashers"] / (
    df["total_busy_dashers"] + df["total_onshift_dashers"] + 1e-5
)

In [ ]:
df["order_intensity"] = df["total_outstanding_orders"] / (
    df["total_busy_dashers"] + 1e-5
)
df["delivery_difficulty"] = (
    df["order_intensity"] * df["estimated_store_to_consumer_driving_duration"]
)

In [ ]:
df["price_range"] = df["max_item_price"] - df["min_item_price"]
df["avg_item_price"] = df["subtotal"] / (df["total_items"] + 1e-5)
df["price_volatility"] = df["price_range"] / (df["avg_item_price"] + 1e-5)

In [ ]:
import numpy as np

df["log_subtotal"] = np.log1p(df["subtotal"])
df["log_outstanding_orders"] = np.log1p(df["total_outstanding_orders"].clip(lower=1e-5))

In [ ]:
df["historical_avg_delivery_time"] = df.groupby(["store_id", "hour"])[
    "delivery_duration_minutes"
].transform("mean")

df["delivery_speed"] = df["historical_avg_delivery_time"] / (
    df["estimated_store_to_consumer_driving_duration"] / 60 + 1e-5
)

In [12]:
df = df.drop(
    columns=["market_id", "created_at", "actual_delivery_time", "store_id"], axis=1
)
df.head()

,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,...,%_dashers_avail,order_intensity,delivery_difficulty,price_range,avg_item_price,price_volatility,log_subtotal,log_outstanding_orders,historical_avg_delivery_time,delivery_speed
0,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,...,0.666664,0.999995,689.996550,0,1899.981000,0.000000,7.550135,1.098612,59.158333,5.144198
1,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,...,0.000000,0.000000,0.000000,0,1899.981000,0.000000,7.550135,0.000010,34.008333,2.957244
2,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,...,0.499998,1.999980,577.994220,1200,1149.998083,1.043480,8.839422,1.098612,59.158333,12.281981
3,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,...,0.500000,1.499998,974.998375,500,1299.995667,0.384617,8.268988,2.302585,33.116667,3.056920
4,NaN,1.0,3,5000,3,1500,1900,2.0,2.0,2.0,...,0.499999,0.999995,337.998310,400,1666.661111,0.240001,8.517393,1.098612,34.008333,6.036972


In [13]:
df.nunique()

store_primary_category                              74
order_protocol                                       7
total_items                                         57
subtotal                                          8368
num_distinct_items                                  20
min_item_price                                    2312
max_item_price                                    2652
total_onshift_dashers                              168
total_busy_dashers                                 154
total_outstanding_orders                           281
estimated_order_place_duration                      98
estimated_store_to_consumer_driving_duration      1336
delivery_duration_minutes                         7134
hour                                                19
day_of_week_num                                      7
is_weekend                                           2
dashers_per_order                                 8629
%_dashers_avail                                   6454
order_inte

In [14]:
numerical_cols = df.select_dtypes(include=["int64", "float64"]).columns
categorical_cols = df.select_dtypes(include=["object", "category"]).columns

In [15]:
df["delivery_duration_minutes"].isnull().sum()

7

In [16]:
df.dropna(inplace=True)

In [17]:
df["dasher_latency_rate"] = df["total_busy_dashers"] / df["total_onshift_dashers"]
df["delay_time"] = (
    df["estimated_store_to_consumer_driving_duration"]
    + df["estimated_order_place_duration"]
)
df.drop(
    inplace=True,
    axis=1,
    columns=[
        "total_busy_dashers",
        "total_onshift_dashers",
        "estimated_store_to_consumer_driving_duration",
        "estimated_order_place_duration",
    ],
)
df.shape

(176015, 25)

In [18]:
def remove_outliers_iqr(df):
    df_cleaned = df.copy()

    for column in df_cleaned.select_dtypes(include=["float64", "int64"]).columns:
        Q1 = df_cleaned[column].quantile(0.25)
        Q3 = df_cleaned[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_cleaned = df_cleaned[
            (df_cleaned[column] >= lower_bound) & (df_cleaned[column] <= upper_bound)
        ]

    return df_cleaned


df_cleaned = remove_outliers_iqr(df)

In [19]:
numerical_cols = df_cleaned.select_dtypes(include=["int64", "float64"]).columns
categorical_cols = df_cleaned.select_dtypes(include=["object", "category"]).columns

In [20]:
def calculate_category_percentages(df):
    results = []
    for column in df.select_dtypes(include=["object"]).columns:
        category_counts = df[column].value_counts(normalize=True) * 100
        percentage_df = category_counts.reset_index()
        percentage_df.columns = ["Category", "Percentage"]
        percentage_df["Column"] = column
        results.append(percentage_df)
    final_result = pd.concat(results, ignore_index=True)
    return final_result


category_percentages = calculate_category_percentages(df_cleaned)

print("Percentage of Each Class in All Categorical Columns:")
category_percentages.head(40)

Percentage of Each Class in All Categorical Columns:


,Category,Percentage,Column
0,american,10.098427,store_primary_category
1,mexican,9.920697,store_primary_category
2,pizza,7.881471,store_primary_category
3,burger,6.406619,store_primary_category
4,sandwich,5.876545,store_primary_category
5,chinese,4.974380,store_primary_category
6,dessert,4.587738,store_primary_category
7,japanese,3.954767,store_primary_category
8,thai,3.814453,store_primary_category
9,vietnamese,3.714675,store_primary_category


## Train Test Split

In [22]:
X = df_cleaned.drop(["delivery_duration_minutes"], axis=1)
y = df_cleaned["delivery_duration_minutes"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [24]:
categorical_features = ["store_primary_category"]
numerical_features = [
    "order_protocol",
    "total_items",
    "subtotal",
    "num_distinct_items",
    "min_item_price",
    "max_item_price",
    "total_outstanding_orders",
    "dasher_latency_rate",
    "delay_time",
    "hour",
    "day_of_week_num",
    "is_weekend",
    "dashers_per_order",
    "%_dashers_avail",
    "order_intensity",
    "delivery_difficulty",
    "price_range",
    "avg_item_price",
    "price_volatility",
    "log_subtotal",
    "log_outstanding_orders",
    "historical_avg_delivery_time",
    "delivery_speed",
]

## Preprocessor

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

preprocessor = ColumnTransformer(
    transformers=[
        (
            "num",
            Pipeline(
                steps=[
                    ("imputer", SimpleImputer(strategy="mean")),
                    ("scaler", StandardScaler()),
                    ("pca", PCA(n_components=0.95)),
                ]
            ),
            numerical_features,
        ),
        (
            "cat",
            OneHotEncoder(handle_unknown="ignore", sparse_output=False),
            categorical_features,
        ),
    ]
)

## Model Training

In [ ]:
from sklearn.linear_model import ElasticNet

elasticnet_model = ElasticNet()

pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", elasticnet_model)])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)


from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R² Score: {r2:.2f}")

Mean Absolute Error (MAE): 8.44
Root Mean Squared Error (RMSE): 10.71
R² Score: 0.37


In [ ]:
from sklearn.model_selection import GridSearchCV

elasticnet_model = ElasticNet()

pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", elasticnet_model)])

grid_params = {
    "model__alpha": [0.01, 0.1, 0.5, 1],
    "model__l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9],
    "model__max_iter": [500, 1000],
    "model__tol": [0.001, 0.01],
}

grid_search = GridSearchCV(pipeline, grid_params, cv=5, scoring="r2", n_jobs=-1)
grid_search.fit(X_train, y_train)

y_pred = grid_search.best_estimator_.predict(X_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Best Parameters: {grid_search.best_params_}")
print("------------------------------")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R² Score: {r2:.2f}")

Best Parameters: {'model__alpha': 0.01, 'model__l1_ratio': 0.1, 'model__max_iter': 500, 'model__tol': 0.01}
------------------------------
Mean Absolute Error (MAE): 7.97
Root Mean Squared Error (RMSE): 10.27
R² Score: 0.42


## Custom Prediction

In [40]:
def predict_delivery_time(input_data):
    """
    Predict the delivery time in minutes for given input features.

    Parameters:
    input_data (dict): A dictionary containing the feature values.

    Returns:
    float: Predicted delivery time in minutes.
    """
    input_df = pd.DataFrame([input_data])
    input_df['price_range'] = input_df['max_item_price'] - input_df['min_item_price']
    input_df['is_weekend'] = input_df['day_of_week_num'].isin([5, 6]).astype(int)
    input_df['order_intensity'] = input_df['total_outstanding_orders'] / (input_df['total_busy_dashers'] + 1e-5)
    input_df['log_subtotal'] = np.log1p(input_df['subtotal'])
    input_df['log_outstanding_orders'] = np.log1p(input_df['total_outstanding_orders'].clip(lower=1e-5))
    input_df['avg_item_price'] = input_df['subtotal'] / (input_df['total_items'] + 1e-5)
    input_df['price_volatility'] = input_df['price_range'] / (input_df['avg_item_price'] + 1e-5)

    prediction = grid_search.predict(input_df)
    return prediction[0]

In [ ]:
example_input = {
    "store_primary_category": "american", 
    "order_protocol": 1.0, # 1 - 7                 
    "total_items": 3, # 1 - 7
    "subtotal": 400, # 400 - 6000
    "num_distinct_items": 2, # 1 - 6
    "min_item_price": 5, # 0 - 1999
    "max_item_price": 50, # 100 - 2000
    "total_outstanding_orders": 3.0, # 3 - 180
    "dasher_latency_rate": 0.5, # 0.5 - 1.25
    "delay_time": 250, # 250 - 1500
    "hour": 14, # 0 - 23
    "day_of_week_num": 0, # 0 - 6
    "dashers_per_order": 0.4, # 0.4 - 1.4
    "%_dashers_avail": 0.4, # 0.4 - 0.6
    "total_busy_dashers": 2, # 0 - 150
    "delivery_difficulty": 100, # 100 - 1500
    "historical_avg_delivery_time": 20, # 20 - 75
    "delivery_speed": 5, # 1 - 12
}

predicted_time = predict_delivery_time(example_input)
print(f"Predicted Delivery Time: {predicted_time:.2f} minutes")

Predicted Delivery Time: 17.18 minutes
